In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-file/idk.json
/kaggle/input/subset/idk.ipynb
/kaggle/input/subset/images/images/cam_1532402933112460.jpg
/kaggle/input/subset/images/images/cam_1532402935662460.jpg
/kaggle/input/subset/images/images/cam_1532402938612460.jpg
/kaggle/input/subset/images/images/cam_1533151614412404.jpg
/kaggle/input/subset/images/images/cam_1533151621412404.jpg
/kaggle/input/subset/images/images/cam_1533151620412404.jpg
/kaggle/input/subset/images/images/cam_1533151609512404.jpg
/kaggle/input/subset/images/images/cam_1532402943662460.jpg
/kaggle/input/subset/images/images/cam_1532402941262460.jpg
/kaggle/input/subset/images/images/cam_1533151619412404.jpg
/kaggle/input/subset/images/images/cam_1532402934662484.jpg
/kaggle/input/subset/images/images/cam_1532402940762460.jpg
/kaggle/input/subset/images/images/cam_1533151613362404.jpg
/kaggle/input/subset/images/images/cam_1532402929662460.jpg
/kaggle/input/subset/images/images/cam_1532402942162460.jpg
/kaggle/input/subset/images/image

In [3]:
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 4.3 MB/s eta 0:00:00a 0:00:01


In [4]:
!pip install einops
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 7.3 MB/s eta 0:00:00ta 0:00:01


In [5]:
 # !mkdir -p /data/sets/nuscenes  # Make the directory to store the nuScenes dataset in.

 # !wget https://www.nuscenes.org/data/v1.0-mini.tgz  # Download the nuScenes mini split.

 # !tar -xf v1.0-mini.tgz -C /data/sets/nuscenes  # Uncompress the nuScenes mini split.

 # !pip install nuscenes-devkit &> /dev/null  # Install nuScenes.

In [6]:
import os
import json
import torch
import torch.multiprocessing as mp
import argparse
import csv
from PIL import Image
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoProcessor
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import sys
from transformers import (
    AdamW,
    AutoModelForCausalLM,
    AutoProcessor,
    get_scheduler
)
from tqdm import tqdm
from typing import List, Dict, Any, Tuple, Generator
from peft import LoraConfig, get_peft_model
from PIL import Image

In [7]:
directory_name = '/kaggle/working/datasett'
try:
    os.mkdir(directory_name)
    print(f"Directory '{directory_name}' created successfully.")
except FileExistsError:
    print(f"Directory '{directory_name}' already exists.")
except PermissionError:
    print(f"Permission denied: Unable to create '{directory_name}'.")
except Exception as e:
    print(f"An error occurred: {e}")

Directory '/kaggle/working/datasett' created successfully.


In [8]:
# from transformers import AutoModelForCausalLM, AutoProcessor

CHECKPOINT = "microsoft/Florence-2-base-ft"
REVISION = 'refs/pr/6'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    CHECKPOINT, trust_remote_code=True, revision=REVISION).to(DEVICE)
processor = AutoProcessor.from_pretrained(
    CHECKPOINT, trust_remote_code=True, revision=REVISION)

config.json:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


pytorch_model.bin:   0%|          | 0.00/464M [00:00<?, ?B/s]

...

processing_florence2.py:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


# Test Inference

In [9]:
# import supervision as sv
# from PIL import Image

# image = Image.open(EXAMPLE_IMAGE_PATH)
# task = "<OD>"
# text = "<OD>"

# inputs = processor(
#     text=text, 
#     images=image, 
#     return_tensors="pt"
# ).to(DEVICE)
# generated_ids = model.generate(
#     input_ids=inputs["input_ids"],
#     pixel_values=inputs["pixel_values"],
#     max_new_tokens=1024,
#     num_beams=3)
# generated_text = processor.batch_decode(
#     generated_ids, skip_special_tokens=False)[0]
# response = processor.post_process_generation(
#     generated_text, 
#     task=task, 
#     image_size=image.size)
# detections = sv.Detections.from_lmm(
#     sv.LMM.FLORENCE_2, response, resolution_wh=image.size)

# bounding_box_annotator = sv.BoundingBoxAnnotator(
#     color_lookup=sv.ColorLookup.INDEX)
# label_annotator = sv.LabelAnnotator(
#     color_lookup=sv.ColorLookup.INDEX)

# image = bounding_box_annotator.annotate(image, detections)
# image = label_annotator.annotate(image, detections)

In [10]:
# import supervision as sv
# from PIL import Image

# image = Image.open(EXAMPLE_IMAGE_PATH)
# task = "<CAPTION_TO_PHRASE_GROUNDING>"
# text = "<CAPTION_TO_PHRASE_GROUNDING> In this image we can see a person wearing a bag and holding a dog. In the background there are buildings, poles and sky with clouds."

# inputs = processor(
#     text=text, 
#     images=image, 
#     return_tensors="pt"
# ).to(DEVICE)
# generated_ids = model.generate(
#     input_ids=inputs["input_ids"],
#     pixel_values=inputs["pixel_values"],
#     max_new_tokens=1024,
#     num_beams=3)
# generated_text = processor.batch_decode(
#     generated_ids, skip_special_tokens=False)[0]
# response = processor.post_process_generation(
#     generated_text, 
#     task=task, 
#     image_size=image.size)
# detections = sv.Detections.from_lmm(
#     sv.LMM.FLORENCE_2, response, resolution_wh=image.size)

# bounding_box_annotator = sv.BoundingBoxAnnotator(
#     color_lookup=sv.ColorLookup.INDEX)
# label_annotator = sv.LabelAnnotator(
#     color_lookup=sv.ColorLookup.INDEX)

# image = bounding_box_annotator.annotate(image, detections)
# image = label_annotator.annotate(image, detections)

In [11]:
from peft import LoraConfig, get_peft_model

TARGET_MODULES = [
    "q_proj", "o_proj", "k_proj", "v_proj", 
    "linear", "Conv2d", "lm_head", "fc2"
]

config = LoraConfig(
    r=8,
    lora_alpha=8,
    target_modules=TARGET_MODULES,
    task_type="CAUSAL_LM",
    lora_dropout=0.05,
    bias="none",
    inference_mode=False,
    use_rslora=True,
    init_lora_weights="gaussian",
    revision=REVISION
)

peft_model = get_peft_model(model, config)
peft_model.print_trainable_parameters()

trainable params: 1,929,928 || all params: 272,733,896 || trainable%: 0.7076


Dataset Formatting

In [12]:
import os
import json
from PIL import Image

def correct_format(image_dir, corrected_data_path):
    # Load the corrected data
    with open(corrected_data_path, 'r') as f:
        corrected_data = json.load(f)

    # Ensure the dataset is in the correct format
    for entry in corrected_data:
        image_path = os.path.join(image_dir,entry['filename'])
        caption = entry['description']

        # Extract the image file name from the image path
        image_file_name = entry['filename']
        json_file_name = os.path.splitext(image_file_name)[0] + '.json'
        json_file_path = os.path.join(image_dir, json_file_name)
        new_json_file_path = os.path.join('/kaggle/working/datasett', json_file_name)
        filename = entry['filename'] # int(image_path.split('/')[-1].split('_')[-2])
        # Split the filename by '_' and take the before last element
        # Create the JSON content
        json_content = {
            "questionId": filename,
            "question": "Describe this image",
            "answers": caption
        }
        # Write the JSON content to the file
        with open(new_json_file_path, 'w') as json_file:
            json.dump(json_content, json_file, indent=4)
    print("Dataset has been formatted correctly.")

In [13]:
image_dir = '/kaggle/input/subset/images/images'
corrected_data_path = '/kaggle/input/cleaned-file/idk.json'
correct_format(image_dir, corrected_data_path)

Dataset has been formatted correctly.


### Downloading the formatted JSONS

In [14]:
# !zip -r file.zip /kaggle/working/jsons

### Copying images to output directory

In [16]:
import os
import shutil

# Source and destination directories
source_dir = '/kaggle/input/subset/images/images'
destination_dir = '/kaggle/working/datasett'

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Copy all image files from source to destination
for file_name in os.listdir(source_dir):
    if file_name.endswith('.jpg') or file_name.endswith('.png'):
        source_file = os.path.join(source_dir, file_name)
        destination_file = os.path.join(destination_dir, file_name)
        shutil.copy(source_file, destination_file)

print("Images have been copied successfully.")

Images have been copied successfully.


## Create Dataset class


In [17]:
class JSONDataset:
    def __init__(self, json_file_path: str, image_directory_path: str):
        self.json_file_path = json_file_path
        self.image_directory_path = image_directory_path
        self.entries = self._load_entries()

    def _load_entries(self) -> List[Dict[str, Any]]:
        entries = []
        with open(self.json_file_path, 'r') as file:
            for line in file:
                data = json.loads(line)
                entries.append(data)
        return entries

    def __len__(self) -> int:
        return len(self.entries)

    def __getitem__(self, idx: int) -> Tuple[Image.Image, Dict[str, Any]]:
        if idx < 0 or idx >= len(self.entries):
            raise IndexError("Index out of range")

        entry = self.entries[idx]
        image_path = os.path.join(self.image_directory_path, entry['image'])
        try:
            image = Image.open(image_path)
            return (image, entry)
        except FileNotFoundError:
            raise FileNotFoundError(f"Image file {image_path} not found.")

In [18]:
class DetectionDataset(Dataset):
    def __init__(self, data):
        self.dataset = data

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        example = self.dataset[idx]
        question = "<DocVQA>" + example['question']
        answers = example['answers']
        if answers is None:
            answers = [""]  # Handle case where answers is None
        elif isinstance(answers, dict):
            answers = list(answers.values())  # Handle case where answers is a dictionary
        elif not isinstance(answers, list):
            answers = [str(answers)]  # Convert answers to a list of strings if not already

        first_answer = answers[0] if answers else ""
        image = example['image']
        if image.mode != "RGB":
            image = image.convert("RGB")
        return question, first_answer, image

In [19]:
def load_local_dataset(folder_name):
    data = []
    for file_name in os.listdir(folder_name):
        
        if file_name.endswith('.json'):
            base_name = os.path.splitext(file_name)[0]
            with open(os.path.join(folder_name, file_name), 'r') as f:
                entry = json.load(f)
                question_id = entry['questionId']
                image_file_name = f'{base_name}.jpg'
                image_path = os.path.join(folder_name, image_file_name)
                if os.path.exists(image_path):
                    try:
                        with open(image_path, 'rb') as img_file:
                            image = Image.open(img_file).convert('RGB')
                            entry['image'] = image
                    except Exception as e:
                        print(f"Error loading image for questionId {question_id}: {e}")
                else:
                    print(f"Image file {image_file_name} not found.")
                data.append(entry)
    return data

def split_data(dataset_folder='datasett', split_ratio=0.8, batch_size=2, num_workers=0):
      data = load_local_dataset(dataset_folder)
      # Check if all entries have images and count them
      train_images_count = sum(1 for entry in data if 'image' in entry)
      print(f"Total images in the dataset: {train_images_count}")

      # Split dataset into training and validation sets
      split_index = int(len(data) * split_ratio)
      train_data = data[:split_index]
      val_data = data[split_index:]

      print(f"Total images in the training set: {sum(1 for entry in train_data if 'image' in entry)}")
      print(f"Total images in the validation set: {sum(1 for entry in val_data if 'image' in entry)}")

      # Create datasets and dataloaders
      train_dataset = DetectionDataset(train_data)
      val_dataset = DetectionDataset(val_data)

      train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=lambda x: collate_fn(x, processor, DEVICE), num_workers=num_workers, shuffle=True)
      val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=lambda x: collate_fn(x, processor, DEVICE), num_workers=num_workers)
      return train_loader, val_loader

In [20]:
train_loader, val_loader = split_data(dataset_folder='/kaggle/working/datasett', split_ratio=0.8, batch_size=2, num_workers=0)

Total images in the dataset: 79
Total images in the training set: 63
Total images in the validation set: 16


# Splitting dataset

In [ ]:
# os.mkdir(f"{destination_dir}/valid")
# os.mkdir(f"{destination_dir}/train")

In [21]:
# @title Initiate `DetectionsDataset` and `DataLoader` for train and validation subsets
BATCH_SIZE = 6
NUM_WORKERS = 0

def collate_fn(batch, processor, device):
    questions, answers, images = zip(*batch)
    inputs = processor(text=list(questions), images=list(images), return_tensors="pt", padding=True).to(device)
    return inputs, answers

# train_dataset = DetectionDataset(
#     json_file_path = f"{destination_dir}/train/captions.json",
#     image_directory_path = f"{destination_dir}/train/"
# )
# val_dataset = DetectionDataset(
#     json_file_path = f"{destination_dir}/valid/captions.json",
#     image_directory_path = f"{destination_dir}/valid/"
# )

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, num_workers=NUM_WORKERS, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, num_workers=NUM_WORKERS)

# Load dataset

In [22]:
def load_local_dataset(folder_name):
    data = []
    for file_name in os.listdir(folder_name):
        if file_name.endswith('.json'):
            base_name = os.path.splitext(file_name)[0]
            with open(os.path.join(folder_name, file_name), 'r') as f:
                entry = json.load(f)
                # question_id = entry['questionId']
                image_file_name = f'{base_name}.jpg'
                image_path = os.path.join(folder_name, image_file_name)
                if os.path.exists(image_path):
                    try:
                        with open(image_path, 'rb') as img_file:
                            image = Image.open(img_file).convert('RGB')
                            entry['image'] = image
                    except Exception as e:
                        print(f"Error loading image for questionId {question_id}: {e}")
                else:
                    print(f"Image file {image_file_name} not found.")
                data.append(entry)
    return data

In [23]:
def train_model(train_loader, val_loader, model, processor, epochs, lr=1e-6):
    optimizer = AdamW(model.parameters(), lr=lr)
    num_training_steps = epochs * len(train_loader)
    lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )

    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for inputs, answers in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{epochs}"):
            input_ids = inputs["input_ids"]
            pixel_values = inputs["pixel_values"]
            labels = processor.tokenizer(
                text=answers,
                return_tensors="pt",
                padding=True,
                return_token_type_ids=False
            ).input_ids.to(DEVICE)
            outputs = model(input_ids=input_ids, pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print(f"Average Training Loss: {avg_train_loss}")

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, answers in tqdm(val_loader, desc=f"Validation Epoch {epoch + 1}/{epochs}"):

                input_ids = inputs["input_ids"]
                pixel_values = inputs["pixel_values"]
                labels = processor.tokenizer(
                    text=answers,
                    return_tensors="pt",
                    padding=True,
                    return_token_type_ids=False
                ).input_ids.to(DEVICE)

                outputs = model(input_ids=input_ids, pixel_values=pixel_values, labels=labels)
                loss = outputs.loss

                val_loss += loss.item()

            avg_val_loss = val_loss / len(val_loader)
            val_losses.append(avg_val_loss)
            print(f"Average Validation Loss: {avg_val_loss}")

        output_dir = f"./model_checkpoints/epoch_{epoch+1}"
        os.makedirs(output_dir, exist_ok=True)
        model.save_pretrained(output_dir)
        processor.save_pretrained(output_dir)

        # save_losses_and_plot(train_losses, val_losses)


In [24]:
train_model(train_loader, val_loader, model, processor, epochs=5, lr=1e-6)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training Epoch 1/5: 100%|██████████| 32/32 [00:29<00:00,  1.10it/s]


Average Training Loss: 5.089716359972954


Validation Epoch 1/5: 100%|██████████| 8/8 [00:03<00:00,  2.21it/s]


Average Validation Loss: 4.573293507099152


Training Epoch 2/5: 100%|██████████| 32/32 [00:28<00:00,  1.10it/s]


...

Average Validation Loss: 4.473590701818466


Training Epoch 5/5: 100%|██████████| 32/32 [00:32<00:00,  1.02s/it]


Average Training Loss: 4.818405136466026


Validation Epoch 5/5: 100%|██████████| 8/8 [00:04<00:00,  1.96it/s]


Average Validation Loss: 4.4673155546188354


In [34]:
output_dir = f"/kaggle/working/model_checkpoint"
os.makedirs(output_dir, exist_ok=True)
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)


[]

## IDK REALLY

Ignore for now

In [ ]:
# # @title Collect predictions

# PATTERN = r'([a-zA-Z0-9 ]+ of [a-zA-Z0-9 ]+)<loc_\d+>'

# def extract_classes(dataset: DetectionDataset):
#     class_set = set()
#     for i in range(len(dataset.dataset)):
#         image, data = dataset.dataset[i]
#         suffix = data["suffix"]
#         classes = re.findall(PATTERN, suffix)
#         class_set.update(classes)
#     return sorted(class_set)

# CLASSES = extract_classes(train_dataset)

# targets = []
# predictions = []

# for i in range(len(val_dataset.dataset)):
#     image, data = val_dataset.dataset[i]
#     prefix = data['prefix']
#     suffix = data['suffix']

#     inputs = processor(text=prefix, images=image, return_tensors="pt").to(DEVICE)
#     generated_ids = model.generate(
#         input_ids=inputs["input_ids"],
#         pixel_values=inputs["pixel_values"],
#         max_new_tokens=1024,
#         num_beams=3
#     )
#     generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

#     prediction = processor.post_process_generation(generated_text, task='<OD>', image_size=image.size)
#     prediction = sv.Detections.from_lmm(sv.LMM.FLORENCE_2, prediction, resolution_wh=image.size)
#     prediction = prediction[np.isin(prediction['class_name'], CLASSES)]
#     prediction.class_id = np.array([CLASSES.index(class_name) for class_name in prediction['class_name']])
#     prediction.confidence = np.ones(len(prediction))

#     target = processor.post_process_generation(suffix, task='<OD>', image_size=image.size)
#     target = sv.Detections.from_lmm(sv.LMM.FLORENCE_2, target, resolution_wh=image.size)
#     target.class_id = np.array([CLASSES.index(class_name) for class_name in target['class_name']])

#     targets.append(target)
#     predictions.append(prediction)

# Run Inference

In [ ]:
# def render_inline(image: Image.Image, resize=(128, 128)):
#     """Convert image into inline html."""
#     image.resize(resize)
#     with io.BytesIO() as buffer:
#         image.save(buffer, format='jpeg')
#         image_b64 = str(base64.b64encode(buffer.getvalue()), "utf-8")
#         return f"data:image/jpeg;base64,{image_b64}"


# def render_example(image: Image.Image, response):
#     try:
#         detections = sv.Detections.from_lmm(sv.LMM.FLORENCE_2, response, resolution_wh=image.size)
#         image = sv.BoundingBoxAnnotator(color_lookup=sv.ColorLookup.INDEX).annotate(image.copy(), detections)
#         image = sv.LabelAnnotator(color_lookup=sv.ColorLookup.INDEX).annotate(image, detections)
#     except:
#         print('failed to redner model response')
#     return f"""
# <div style="display: inline-flex; align-items: center; justify-content: center;">
#     <img style="width:256px; height:256px;" src="{render_inline(image, resize=(128, 128))}" />
#     <p style="width:512px; margin:10px; font-size:small;">{html.escape(json.dumps(response))}</p>
# </div>
# """


# def render_inference_results(model, dataset: DetectionDataset, count: int):
#     html_out = ""
#     count = min(count, len(dataset))
#     for i in range(count):
#         image, data = dataset.dataset[i]
#         prefix = data['prefix']
#         suffix = data['suffix']
#         inputs = processor(text=prefix, images=image, return_tensors="pt").to(DEVICE)
#         generated_ids = model.generate(
#             input_ids=inputs["input_ids"],
#             pixel_values=inputs["pixel_values"],
#             max_new_tokens=1024,
#             num_beams=3
#         )
#         generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
#         answer = processor.post_process_generation(generated_text, task='<OD>', image_size=image.size)
#         html_out += render_example(image, answer)

#     display(HTML(html_out))

# render_inference_results(peft_model, val_dataset, 4)

# Save Model

In [36]:
# os.mkdir('/kaggle/working/model')
# os.mkdir('/kaggle/working/model/florence2-lora')
peft_model.save_pretrained("/model/florence2-lora")
processor.save_pretrained("/model/florence2-lora/")

# !ls -la /content/florence2-lora/

[]

In [33]:
!pip install inference

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
import os
from inference import get_model
from PIL import Image
import json

lora_model = get_model("model-id/version-id", api_key="KEY")

image = Image.open("containers.png")
response = lora_model.infer(image)
print(response)

ModelArtefactError: Error loading model artifacts from Roboflow API.